Скрипт для предсказания категории товара на каждом уровне иерархии.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from pymystem3 import Mystem
import re

import time

В качестве предикторов используются описания товаров, соединенные с их заголовками.

In [2]:
df_categ = pd.read_csv('./data/category.csv', index_col='category_id')
df_train = pd.read_csv('./data/train.csv', index_col='item_id')
df_train = df_train.rename(columns={'description':'text', 'category_id':'cat_id'})
df_train['text'] = (df_train['title'] + ' ') * 2 + df_train['text']
df_train = df_train.drop(['title'], axis=1)

Разбиение категорий на уровни.

In [3]:
cats = [cat.split('|') for cat in df_categ.name]
cats = pd.DataFrame(cats).fillna(value='None')
Y = cats

Для каждого уровня иехархии используется свой LabelEncoder для последующего обращения кодирования.

In [4]:
from sklearn.preprocessing import LabelEncoder
le = []
le.append(LabelEncoder())
Y[0] = le[0].fit_transform(Y[0])

le.append(LabelEncoder())
Y[1] = le[1].fit_transform(Y[1])

le.append(LabelEncoder())
Y[2] = le[2].fit_transform(Y[2])

le.append(LabelEncoder())
Y[3] = le[3].fit_transform(Y[3])

Y['cat_id'] = Y.index

In [5]:
df = df_train.merge(Y, how='left', on='cat_id')

Модель для иерархической класификации реализована в виде класса с двумя интерфейсами:  
fit(df) принимает датафрейм с текстом и номерами категорий, созданный выше и обучает модель;  
predict(X) принимает текст и выдает массив с предсказанными категориями на каждом уровне.

В модели используется отдельный классификатор для каждого уровня иерархии, но для классификации товара на каждом уровне после корневого, к предикторам добавляется предсказанный класс товара на предыдущем уровне.

При создании модель получает массив обьектов типа LabelEncoder.

In [6]:
class HierarchyClassifier():
    def __init__(self, encoders):
        self.model = [
            SGDClassifier(alpha=0.000001, random_state=1, class_weight='balanced', penalty='l2', loss='log', n_jobs=4),
            SGDClassifier(alpha=0.000001, random_state=1, class_weight='balanced', penalty='l2', loss='log', n_jobs=4),
            SGDClassifier(alpha=0.000001, random_state=1, class_weight='balanced', penalty='l2', loss='log', n_jobs=4),
            SGDClassifier(alpha=0.000001, random_state=1, class_weight='balanced', penalty='l2', loss='log', n_jobs=4)
        ]
        self.encoders = encoders
        self.tv = [TfidfVectorizer(min_df=2, ngram_range=(1,3), sublinear_tf=True),
                   TfidfVectorizer(min_df=2, ngram_range=(1,3), sublinear_tf=True),
                   TfidfVectorizer(min_df=2, ngram_range=(1,3), sublinear_tf=True),
                   TfidfVectorizer(min_df=2, ngram_range=(1,3), sublinear_tf=True)]
        
    def clear_text(self, text):
        text = text.lower()
        text = re.sub(r'[^a-zA-Zа-яА-Я- ]', '', text)
        text = re.sub(r' +', ' ', text)
        return text    
        
    def preproc(self, X):
        start_time = time.monotonic()
        lemma = Mystem()
        tokenizer = RegexpTokenizer(r'\w+')
        X = [self.clear_text(x) for x in X]
        dictinar = [tokenizer.tokenize(x) for x in X]
        stop = stopwords.words('russian') + ['продать', 'продаваться', 'продоваться', 'купить', 'продавать', 'продажа',
                  'новое', 'новый', 'хороший', 'отличный', 'состояние', 'идеальный',
                  'торг', 'цена', 'уместный','поэтому', 'также', 'обмен', 'срочно', 'который',
                  'сайт', 'это', 'вопрос', 'любой', 'причина', 'магазин', 'звонить', 'писать',
                  'очень', 'абсолютно', 'назад', 'использоваться', 'практически', 'возможный',
                 ]
        resX = []
        for otz in dictinar:
            otiv = [word for word in otz if word not in stop]
            otiv = [word for word in otiv if not word.isnumeric()]
            otiv = ' '.join(otiv)
            otiv = lemma.lemmatize(otiv)
            otiv = [word for word in otiv if word not in ['\n']]
            otiv = ''.join(otiv)
            resX.append(otiv)
        print('Сompleted ({:.2f} sec)'.format(time.monotonic() - start_time))
        
        return resX   

        
    def make_x_y(self, dfr, level=0, prev_predict=None, percentage=0.8):
        """
        Разбивает датасет на тестовую и обучающую части
        и добавляет к предиторам результаты работы модели на предыдущем уровне
        (в случае не корневого уровня)
        """
        X,Y = (dfr['text'], dfr[level])
        #добавление категорий предыдущего уровня
        if not (prev_predict is None):
            prev_predict = self.encoders[level-1].inverse_transform(prev_predict)
            X = X.apply(lambda x: x+' ') + pd.Series(prev_predict).astype('str')
            
        p = int(len(X) * percentage)
        
        x_train = X[:p]
        x_test = X[p:]
        
        y_train = Y[:p]
        y_test = Y[p:]
        
        self.tv[level].fit(x_train)
        x_train = self.tv[level].transform(x_train)
        x_test = self.tv[level].transform(x_test)
        
        return x_train, y_train, x_test, y_test
    
    
    def fit(self, data):
        """
        Функция для обучения моделей
        pred - результат работы модели на предыдущем уровне
        """
        pred = None
        data['text'] = self.preproc(data['text'])
        for i in range(4):
            x_train, y_train, x_test, y_test = self.make_x_y(data, i, pred)
            self.model[i].fit(x_train, y_train)
            pred = np.append(self.model[i].predict(x_train), self.model[i].predict(x_test))
            print('Train accuracy on level',i,':', self.model[i].score(x_train, y_train),
                  'Test accuracy on level',i,':', self.model[i].score(x_test, y_test))
    
    
    def predict(self, X):
        
        prev_predict = []
        pred = []
        
        X = self.preproc(X)
        
        for i in range(4):
            #добавление категорий предыдущего уровня
            print("Уровень {0} обработан".format(i))
            if len(pred) > 0:
                prev_predict = self.encoders[i-1].inverse_transform(pred[i-1])
                X = pd.Series(X).apply(lambda x: x+' ') 
                X += pd.Series(prev_predict).astype('str')
            #использование модели
            X_vect = self.tv[i].transform(X)
            lvl_predict = self.model[i].predict(X_vect)
            pred.append(lvl_predict)
        #преобразование номеров категорий в строки
        for i in range(4):
            pred[i] = self.encoders[i].inverse_transform(pred[i])
            
        return pred

In [7]:
classifier = HierarchyClassifier(encoders=le)

Обучение классфикатора

In [8]:
classifier.fit(df)

Сompleted (130.54 sec)


/home/john/anaconda3/envs/avito/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Train accuracy on level 0 : 0.998675 Test accuracy on level 0 : 0.9518
Train accuracy on level 1 : 0.998175 Test accuracy on level 1 : 0.9273
Train accuracy on level 2 : 0.993775 Test accuracy on level 2 : 0.8712
Train accuracy on level 3 : 0.99865 Test accuracy on level 3 : 0.97375


Загружаем тестовую выборку и делаем предсказания

In [10]:
df_test = pd.read_csv('./data/test.csv', index_col='item_id')
df_test = df_test.rename(columns={'description':'text'})

df_test['text'] = (df_test['title'] + ' ') * 2 + df_test['text']
df_test = df_test.drop(['title'], axis=1)

In [45]:
predicts = classifier.predict(df_test['text'])

Сompleted (12.74 sec)
Уровень 0 обработан
Уровень 1 обработан
Уровень 2 обработан
Уровень 3 обработан


Точность модели:  
Train accuracy on level 0 : 0.998675 Test accuracy on level 0 : 0.9518  
Train accuracy on level 1 : 0.998175 Test accuracy on level 1 : 0.9273  
Train accuracy on level 2 : 0.993775 Test accuracy on level 2 : 0.8712  
Train accuracy on level 3 : 0.99865 Test accuracy on level 3 : 0.97375